In [343]:
import xmltodict
from bs4 import BeautifulSoup
import pprint
import os
import re
import pandas as pd

In [344]:
image_folder = os.path.join(os.getcwd(),'IMAGE_DATA')

In [345]:
image_root = os.path.join(image_folder,os.listdir(image_folder)[0])

In [346]:
image_root_lst = os.listdir(image_root)

In [347]:

r = re.compile('(Index)')
# re.findall(r,image_root_lst)
idx_file = list(filter(r.findall,image_root_lst))[0]


In [348]:
with open(os.path.join(image_root,idx_file),'r') as f:
    index_xml = f.read()

In [349]:
idx_xml = BeautifulSoup(index_xml,'xml')

In [350]:
# ind_file.find('<Col>'==22)
# ind_file.find_all('Wells', {'Col': [22]})

In [351]:
idx_dict = xmltodict.parse(index_xml)

In [352]:
# pprint.pprint(idx_dict['EvaluationInputData'])

In [353]:
idx_dict['EvaluationInputData'].keys()

dict_keys(['@Version', '@xmlns', '@xmlns:xsd', '@xmlns:xsi', 'User', 'InstrumentType', 'Plates', 'Wells', 'Maps', 'Images'])

In [354]:
mod_well_lst_to_df = []

for cnt in range(len(idx_dict['EvaluationInputData']['Wells']['Well'])):
    well_temp_df = pd.DataFrame.from_dict(idx_dict['EvaluationInputData']['Wells']['Well'][cnt],orient='columns')
    well_temp_df['Image'] = [item.pop('@id') for item in well_temp_df['Image']]
    mod_well_lst_to_df.append(well_temp_df)
    well_table=pd.concat(mod_well_lst_to_df,axis=0)

In [355]:
well_table.reset_index(drop=True,inplace=True)

In [356]:
well_table

,id,Col,Row,SublayoutID,Image
0,0101,1,1,1,0101K1F1P1R1
1,0101,1,1,1,0101K1F1P1R2
2,0101,1,1,1,0101K1F2P1R1
3,0101,1,1,1,0101K1F2P1R2
4,0101,1,1,1,0101K1F3P1R1
...,...,...,...,...,...
2299,0124,24,1,1,0124K1F1P1R2
2300,0124,24,1,1,0124K1F2P1R1
2301,0124,24,1,1,0124K1F2P1R2
2302,0124,24,1,1,0124K1F3P1R1


In [245]:
temp_df['Image'][0].pop('@id')

'0124K1F1P1R1'

In [242]:
temp_df['Image'] = [item.pop('@id') for item in temp_df['Image'] ]

,id,Col,Row,SublayoutID,Image
0,0124,24,1,1,{'@id': '0124K1F1P1R1'}
1,0124,24,1,1,{'@id': '0124K1F1P1R2'}
2,0124,24,1,1,{'@id': '0124K1F2P1R1'}
3,0124,24,1,1,{'@id': '0124K1F2P1R2'}
4,0124,24,1,1,{'@id': '0124K1F3P1R1'}
5,0124,24,1,1,{'@id': '0124K1F3P1R2'}


In [219]:
test_df['Image'] = image_id_lst

ValueError: Length of values (2304) does not match length of index (384)

In [135]:
test_df

,id,Col,Row,SublayoutID,Image
0,0101,1,1,1,0101K1F1P1R1
1,0101,1,1,1,0201K1F1P1R2
2,0101,1,1,1,0301K1F2P1R1
3,0101,1,1,1,0401K1F2P1R2
4,0101,1,1,1,0501K1F3P1R1
5,0101,1,1,1,0601K1F3P1R2


In [ ]:


# # r = re.compile('.*r04c11')
# r = re.compile('(Index)')

# with open (list(filter(r.findall,im_lst))[0],'r') as f:
#     z = f.read()
# z


# import numpy as np
# samp = np.array([21,22,33,4,5,6,7,87])
# mask = np.array([0,0,0,1,1,1,1,0],dtype=bool)
# samp[mask]

